In [2]:
import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier as DTC, export_graphviz
import pydot
import pydotplus

import time

In [3]:
Train_data = pd.read_csv('./datalab/car_info_train.csv')
Train_data.info()


FileNotFoundError: [Errno 2] File ./data/car_info_train.csv does not exist: './data/car_info_train.csv'

In [4]:
numerical_cols = Train_data.select_dtypes(exclude = 'object').columns
print(numerical_cols)

NameError: name 'Train_data' is not defined

In [5]:
Train_data['CUST_AGE'] = pd.qcut(Train_data.CUST_AGE, 3, labels=["1", "2", "3"])
Train_data['CAR_AGE'] = pd.qcut(Train_data.CAR_AGE, 5, labels=["1", "2", "3", "4", "5"])
Train_data['CAR_PRICE'] = pd.qcut(Train_data.CAR_PRICE, 4, labels=["1", "2", "3", "4"])
Train_data['LOAN_AMOUNT'] = pd.qcut(Train_data.LOAN_AMOUNT, 4, labels=["1", "2", "3", "4"])

NameError: name 'Train_data' is not defined

In [6]:
Train_data.head()

NameError: name 'Train_data' is not defined

In [7]:
Train_data.isnull().any()

NameError: name 'Train_data' is not defined

In [8]:
feature_cols = [col for col in numerical_cols if col != 'IS_LOST']
X_data = Train_data[feature_cols]
Y_data = Train_data['IS_LOST']
X_data


NameError: name 'numerical_cols' is not defined

In [9]:
dtc = DTC(criterion='entropy',max_depth=4)
dtc.fit(X_data, Y_data)
print('准确率:', dtc.score(X_data, Y_data))
with open('./tree.dot', 'w') as f:
    f = export_graphviz(dtc, feature_names=X_data.columns, out_file=f)

NameError: name 'X_data' is not defined

In [10]:
import pydot
import pydotplus
graph = pydotplus.graph_from_dot_file('./tree.dot')

FileNotFoundError: [Errno 2] No such file or directory: './tree.dot'